# Import libraries

In [71]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [73]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)
mnist_train , mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_validation_samples, tf.int64)

def scale (image,label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image,label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

#Shuffle data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

# Model

# Outline the model

In [90]:
input_size = 784
output_size = 10
hidden_layer_size = 200
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28,28,1)), tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), 
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), tf.keras.layers.Dense(output_size, activation = 'softmax')])

# Choose the optimizer and the loss function

In [92]:
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Traning

In [96]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 10s - loss: 0.2726 - accuracy: 0.9220 - val_loss: 0.1490 - val_accuracy: 0.9578 - 10s/epoch - 18ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1088 - accuracy: 0.9676 - val_loss: 0.0967 - val_accuracy: 0.9692 - 5s/epoch - 9ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0714 - accuracy: 0.9787 - val_loss: 0.0618 - val_accuracy: 0.9793 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0531 - accuracy: 0.9835 - val_loss: 0.0562 - val_accuracy: 0.9820 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0414 - accuracy: 0.9874 - val_loss: 0.0558 - val_accuracy: 0.9827 - 4s/epoch - 8ms/step


# Test the model 

In [88]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

2/2 [==============================] - 1s 175ms/step - loss: 0.1021 - accuracy: 0.9694
Test loss: 0.10. Test accuracy: 96.94%
